In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pyccl as ccl
import scipy
import pandas
from astropy.cosmology import FlatLambdaCDM
import astropy.units as u
from cdfvi import funcs, SNCosmoChi2, SNPolyChi2

In [ ]:
cosmo = ccl.Cosmology(
    Omega_c=0.25, Omega_b=0.05,
    h=0.7, n_s=0.95, sigma8=0.8,
    transfer_function='bbks'
)

cosmo_astro = FlatLambdaCDM(H0=70*u.km / u.s / u.Mpc, Om0=0.3, Ob0=0.05)


zmin = 0.01
zmax = 6
nzBins = 300
z_grid = np.linspace(zmin, zmax, nzBins)
z_widths = z_grid[1:] - z_grid[0:-1]
a_grid = funcs.z_to_a(z_grid)

In [ ]:
param_names = ['H0', 'Om0', 'Ob0']
init_pars_cosmo = [70, 0.25, 0.05]
param_scales = dict(H0=u.km / u.s / u.Mpc)



In [ ]:
dd_obs = pandas.read_csv('DES-SN5YR_HD.csv')

In [ ]:
def simSN(cosmo, dd):
    mu_cosmo = cosmo.distance_modulus(1/(1+dd.zHD.values))
    mu_obs = mu_cosmo + np.random.normal(0, dd.MUERR_FINAL)
    dd_out = dd.copy(deep=True)
    dd_out['MU'] = mu_obs 
    return dd_out

In [ ]:
dd_sim = simSN(cosmo, dd_obs)
dd = dd_sim

In [ ]:
dd

In [ ]:
dd['D_l'] = np.power(10, (dd.MU+5)/5.) / 1e9
dd['D_l_err'] = dd.D_l*dd.MUERR_FINAL
dd['a'] = 1/(1 + dd.zHD)
dd['chi'] = dd.D_l * dd.a
dd['chi_err'] = dd.chi*dd.MUERR_FINAL
mask = dd.MUERR_FINAL < 0.5
ddmask = dd[mask]

In [ ]:
_ = plt.hist(dd.zHD, bins=np.linspace(0, 1.2, 121))

In [ ]:
_ = plt.errorbar(ddmask.zHD, ddmask.MU, ddmask.MUERR_FINAL, ls="", marker='.')
_ = plt.plot(z_grid, cosmo.distance_modulus(a_grid), label=f"$\Lambda$CDM")
_ = plt.xlim(0, 1.2)
_ = plt.ylim(34, 46)
_ = plt.xlabel("z")
_ = plt.ylabel("M - m [mag]")
_ = plt.legend()

In [ ]:
_ = plt.errorbar(ddmask.zHD, ddmask.D_l, ddmask.D_l_err, ls="", marker='.', alpha=0.2)
_ = plt.plot(z_grid, cosmo.luminosity_distance(a_grid)/1e3, label=f"$\Lambda$CDM")
_ = plt.plot(z_grid, cosmo_astro.luminosity_distance(z_grid)/1e3, label=f"$\Lambda$CDM")
_ = plt.xlim(0, 1.2)
_ = plt.ylim(0, 10)
_ = plt.xlabel("z")
_ = plt.ylabel(r"$D_l$ [Gpc]")
_ = plt.legend()

In [ ]:
_ = plt.errorbar(ddmask.zHD, ddmask.chi, ddmask.chi_err, ls="", marker='.', alpha=0.2)
_ = plt.plot(z_grid, cosmo.comoving_radial_distance(a_grid)/1e3, label=r"$\Lambda$CMD")
_ = plt.plot(z_grid, cosmo_astro.comoving_distance(z_grid)/1e3, label=f"$\Lambda$CDM")
_ = plt.xlim(0, 1.1)
_ = plt.ylim(0.1, 10)
_ = plt.yscale('log')
_ = plt.xlabel("z")
_ = plt.ylabel(r"$\chi$ [Gpc]")
_ = plt.legend()

In [ ]:
fixed_vals = dict()
param_names = ['H0', 'Om0', 'Ob0']
init_pars_cosmo = [70, 0.25, 0.05]
param_scales = dict(H0=u.km / u.s / u.Mpc)
sncf = SNCosmoChi2(ddmask.zHD.values, ddmask.chi.values, ddmask.chi_err.values, param_names, param_scales, **fixed_vals)

In [ ]:
#result_c = scipy.optimize.minimize(sncf, init_pars_cosmo, method='nelder-mead', options={'xatol': 1e-9, 'disp': True, 'maxiter':2000})
result_c = scipy.optimize.minimize(sncf, init_pars_cosmo, method='L-BFGS-B', options={'gtol': 0.01})


In [ ]:
_ = plt.errorbar(ddmask.zHD, ddmask.chi, ddmask.chi_err, ls="", marker='.', alpha=0.2)
_ = plt.plot(z_grid, cosmo.comoving_radial_distance(a_grid)/1e3, label=r"$\Lambda$CMD")
_ = plt.plot(z_grid, sncf.modelSpace(z_grid, result_c.x), label='Fit')
_ = plt.xlim(0, 1.1)
_ = plt.ylim(0.1, 4)
#_ = plt.yscale('log')
_ = plt.legend()

In [ ]:
delta = 1 - 1e3*sncf.modelSpace(z_grid, result_c.x)/cosmo.comoving_radial_distance(a_grid)
delta_2= 1 - (1e3*sncf.modelSpace(z_grid, result_c.x)/cosmo_astro.comoving_distance(z_grid)).value

_ = plt.plot(z_grid, delta, label=r"$\Lambda$CMD")
_ = plt.plot(z_grid, delta_2, label=r"$\Lambda$CMD")


In [ ]:
result_c.x

In [ ]:
mat = np.matrix(result_c.hess_inv.todense())
hesse = mat.getI()
h0_grid = np.linspace(60, 80, 401)
Om0_grid = np.linspace(0.2, 0.5, 501)
mesh = np.meshgrid(h0_grid, Om0_grid)
dx, dy = mesh[0] - result_c.x[0], mesh[1] - result_c.x[1]
nulls = np.zeros(dx.shape)
dels = np.squeeze(np.dstack([dx.ravel(), dy.ravel(), nulls.ravel()]))
diffs = np.reshape(np.array([np.matmul(del_, np.matmul(hesse, del_).T) for del_ in dels]), dx.shape)

In [ ]:
_ = plt.contour(h0_grid, Om0_grid, diffs, levels=[1, 4, 9])
_ = plt.xlabel(r'$H_{0} [km \cdot s^{-1} \cdot Mpc^{-1}]$') 
_ = plt.ylabel(r'$\Omega_{m, 0}$')

In [ ]:
poly_class = np.polynomial.Polynomial

In [ ]:
result1d = poly_class.fit(z_grid[1:], funcs.J_integral(cosmo, z_grid), deg=3).convert()
int_j_func = poly_class(result1d.coef)
init_pars = result1d.coef

In [ ]:
_ = plt.plot(z_grid[1:], funcs.J_integral(cosmo, z_grid))
_ = plt.plot(z_grid, result1d(z_grid))
_ = plt.plot(z_grid, int_j_func(z_grid))

In [ ]:
snpf = SNPolyChi2(ddmask.zHEL.values, ddmask.chi.values, ddmask.chi_err.values, constraint=1e-4, poly_type=poly_class)

In [ ]:
result = scipy.optimize.minimize(snpf, init_pars, method='nelder-mead', options={'xatol': 1e-9, 'disp': True, 'maxiter':2000})

In [ ]:
_ = plt.errorbar(ddmask.zHEL, ddmask.chi, ddmask.chi_err, ls="", marker='.', alpha=0.2)
_ = plt.plot(z_grid, snpf.modelSpace(z_grid, result.x, poly_type=poly_class))
_ = plt.xlim(0, 1.1)
_ = plt.ylim(0.1, 10)